# Requiment 1

## Library

In [1]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import hashlib
import random
from sklearn.metrics.pairwise import cosine_similarity
import time

## Data

In [2]:
def read_file(filename):
      with open(filename, 'r') as file:
          text = file.readlines()
      df = pd.DataFrame(text, columns=["Text"])
      return df

## LSH

### Version 1:

In [3]:
class InMemoryMinHashLSH:

    def __init__(self, documents):
      self.documents = documents
    pass

    def convert_to_shingling(self, text, k):
      shingles = set()
      words = text.split(" ")
      for i in range(len(words) - k + 1):
          tmp = " ".join(words[i:i+k])
          shingles.add(tmp)
      return shingles

    def convert_to_boolean_vector(self, df, text):
      union_shingles = set()  # Initialize as a set
      for _, row in df.iterrows():
          union_shingles |= set(row['Shingles'])  # Use set union operation
      # Convert the set to a numpy array for efficient membership checking
      union = np.array(list(union_shingles))
      result = []
      for shingle in union:
          if shingle in text:
              result.append(1)
          else:
              result.append(0)
      return result


    def shingling(self,df):
      shingles_list = []
      boolean_list = []

      for index, row in df.iterrows():
          text = row['Text'].replace(',',"").replace(';',"").replace('.',"").replace('\'',"")
          shingles = self.convert_to_shingling(text, k=8)  # Adjust k value as needed
          shingles_list.append(shingles)
      shingles_df = pd.DataFrame({'Shingles': shingles_list})

      for index, row in df.iterrows():
          text = row['Text'].replace(',',"").replace(';',"").replace('.',"").replace('\'',"")
          boolean_vector = self.convert_to_boolean_vector(shingles_df,text)  # Adjust k value as needed
          boolean_list.append(boolean_vector)
      boolean_vector_df = pd.DataFrame({'vector': boolean_list})
      return boolean_vector_df

    def convert_to_signature(self,text):
      vector = np.array(text)
      arr = self.generate_arrHash(len(vector))
      result=[]
      for i in range(1,50):
        random.shuffle(arr)
        for j in range(1,len(arr)):
          if vector[arr.index(j)]==1:
            result.append(arr.index(j))
            break
      return np.array(result)

    def minhashing(self,df):
      signature_list=[]
      for index, row in df.iterrows():
        text = row['vector']
        minhash = self.convert_to_signature(text)
        signature_list.append(minhash)
      signature_df = pd.DataFrame({'signature': signature_list})

      return signature_df

    def generate_arrHash(self,n):
      arr = [i for i in range(1, n+1)]
      return arr

    def split_signature(self, signature, k):
        result = []
        for i in range(0, len(signature), k):
            tmp = signature[i:i+k].tolist()
            result.append(tmp)
        return result
    # def hash_busket(self,signature_split):


    def truncated_hash(self, vector, length):
        hash_value = hashlib.sha256(bytes(str(vector), 'utf-8')).hexdigest()
        truncated_value = int(hash_value[:length], 16)
        result = {truncated_value: vector}
        return result

    def locality_sensitivity_hashing(self, signature):
        result = {}
        for index, row in signature.iterrows():
            text = row['signature']
            for vector in self.split_signature(text, 2):
                truncated_hash_result = self.truncated_hash(vector, 4)
                for key, value in truncated_hash_result.items():
                    if key not in result:
                        result[key] = []
                    result[key].append(value)
            # break
        return result

    def check_bucket(self,bucket):
      for i in bucket.values():
        if len(i) > 1:
            print(i)

    def run(self):
      shingling_result= self.shingling(self.documents)
      # sig = self.minhashing(shingling_result)
      # result = self.locality_sensitivity_hashing(sig)
      # self.check_bucket(result)
      return


    def approxNearestNeighbors(self, key, n):
        pass


### Version 2:

In [5]:
class InMemoryMinHashLSH:

    def __init__(self, documents):
      self.documents = documents
    pass

    def convert_to_shingling(self, text, k):
      shingles = set()
      words = text.split(" ")
      for i in range(len(words) - k + 1):
          tmp = " ".join(words[i:i+k])
          shingles.add(tmp)
      return shingles


    def convert_to_boolean_vector(self, df, text):
      union_shingles = set().union(*df['Shingles'])
      union = np.array(list(union_shingles))
      result = np.zeros(len(union), dtype=int) # Initialize result as a NumPy array of zeros
      for i, shingle in enumerate(union):
          if shingle in text:
              result[i] = 1
      return result


    def shingling(self, df):
      def preprocess_text(text):
          return text.replace(',', "").replace(';', "").replace('.', "").replace('\'', "")
      df['Text'] = df['Text'].apply(preprocess_text)

      shingles_df = pd.DataFrame({'Shingles': df['Text'].apply(lambda text: self.convert_to_shingling(text, k=8))})
      boolean_vector_df = pd.DataFrame({'vector': df['Text'].apply(lambda text: self.convert_to_boolean_vector(shingles_df, text))})

      return boolean_vector_df


    def convert_to_signature(self, text):
      vector = np.array(text)
      arr = self.generate_arr_hash(len(vector)-1)
      result_list = []  # Use a list for intermediate storage
      arr_set = set(arr)  # Convert arr to a set for faster lookup

      for i in range(1, 50):
          np.random.shuffle(arr)  # Shuffle the array using numpy
          for j in range(len(arr)):
            if vector[arr[j]] == 1:
                result_list.append(arr[j])  # Append to the list
                break
      return np.array(result_list)  # Convert the list to a numpy array

    def minhashing(self,df):
      signature_list=[]
      for index, row in df.iterrows():
        text = row['vector']
        minhash = self.convert_to_signature(text)
        signature_list.append(minhash)
      signature_df = pd.DataFrame({'signature': signature_list})

      return signature_df

    def generate_arr_hash(self,n):
      arr = [i for i in range(1, n+1)]
      return arr

    def split_signature(self, signature, k):
        result = []
        for i in range(0, len(signature), k):
            tmp = signature[i:i+k].tolist()
            result.append(tmp)
        return result
    # def hash_busket(self,signature_split):


    def truncated_hash(self, vector, length):
        hash_value = hashlib.sha256(bytes(str(vector), 'utf-8')).hexdigest()
        truncated_value = int(hash_value[:length], 16)
        result = {truncated_value: vector}
        return result

    def locality_sensitivity_hashing(self, signature):
      result = {}

      def process_row(row):
          text = row['signature']
          hash_results = {}  # Change to a dictionary
          for vector in self.split_signature(text, 2):
              truncated_hash_result = self.truncated_hash(vector, 4)
              for key, value in truncated_hash_result.items():
                  hash_results.setdefault(key, []).extend(value)
          return hash_results

      # Apply the process_row function to each row of the DataFrame
      hash_results_list = signature.apply(process_row, axis=1)

      # Aggregate the hash results into the final result dictionary
      for hash_results in hash_results_list:
          for key, value in hash_results.items():
              result.setdefault(key, []).extend(value)

      return result

    def check_bucket(self,bucket):
      for i in bucket.values():
        if len(i) > 3:
            print(i)

    def run(self):

      ###Shingling
      start_time = time.time()
      shingling_result= self.shingling(self.documents)
      end_time = time.time()
      print("part 1(shingling): ", end_time - start_time)

      ###Minhashing
      start_time = time.time()
      sig = self.minhashing(shingling_result)
      end_time = time.time()
      print("part 2(minhashing): ", end_time - start_time)


      ###LSH
      start_time = time.time()
      result = self.locality_sensitivity_hashing(sig)
      end_time = time.time()
      print("part 3(LSH): ", end_time - start_time)

      return


    def approxNearestNeighbors(self, key, n):
        pass


### Running

In [ ]:
data = read_file("/content/WebOfScience-5736.txt")
lsh = InMemoryMinHashLSH(data)
lsh.run()

part 1(shingling):  2694.1030468940735
